## Map data
heatmap by counties, weights as pollution data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import json
import copy
from typing import List
from urllib.request import urlretrieve
from urllib.parse import quote
import csv
import gmaps
import os
from config import g_key
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Import API key
from config import key
from config import email_key

In [2]:
# Import from .CSV
california_df = pd.read_csv("california_data.csv") 

In [3]:
california_df.head()

,state_code,state,county_code,county,site_number,parameter,sample_measurement,units_of_measure,latitude,longitude,date_local,time_local,date_gmt,time_gmt,method_type,method_code,method,date_of_last_change,cbsa_code
0,6,California,37,Los Angeles,1201,Acceptable PM2.5 AQI & Speciation Mass,5.8,Micrograms/cubic meter (LC),34.19925,-118.53276,2019-01-01,00:00,2019-01-01,08:00,Non-FRM,731,Met-One BAM-1020 W/PM2.5 SCC - Beta Attenuation,2019-06-24,31080
1,6,California,37,Los Angeles,1201,Acceptable PM2.5 AQI & Speciation Mass,4.6,Micrograms/cubic meter (LC),34.19925,-118.53276,2019-01-01,01:00,2019-01-01,09:00,Non-FRM,731,Met-One BAM-1020 W/PM2.5 SCC - Beta Attenuation,2019-06-24,31080
2,6,California,37,Los Angeles,1201,Acceptable PM2.5 AQI & Speciation Mass,2.8,Micrograms/cubic meter (LC),34.19925,-118.53276,2019-01-01,02:00,2019-01-01,10:00,Non-FRM,731,Met-One BAM-1020 W/PM2.5 SCC - Beta Attenuation,2019-06-24,31080
3,6,California,37,Los Angeles,1201,Acceptable PM2.5 AQI & Speciation Mass,4.8,Micrograms/cubic meter (LC),34.19925,-118.53276,2019-01-01,03:00,2019-01-01,11:00,Non-FRM,731,Met-One BAM-1020 W/PM2.5 SCC - Beta Attenuation,2019-06-24,31080
4,6,California,37,Los Angeles,1201,Acceptable PM2.5 AQI & Speciation Mass,3.0,Micrograms/cubic meter (LC),34.19925,-118.53276,2019-01-01,04:00,2019-01-01,12:00,Non-FRM,731,Met-One BAM-1020 W/PM2.5 SCC - Beta Attenuation,2019-06-24,31080


In [4]:
# Make copy before filtering
california_filtered_df = california_df.copy()

In [5]:
# Select certain columns to use for mapping
california_filtered_df=california_filtered_df[['county','site_number','sample_measurement','latitude','longitude','time_local','date_gmt','time_gmt','date_of_last_change']]
california_filtered_df.head(20)

,county,site_number,sample_measurement,latitude,longitude,time_local,date_gmt,time_gmt,date_of_last_change
0,Los Angeles,1201,5.8,34.19925,-118.53276,00:00,2019-01-01,08:00,2019-06-24
1,Los Angeles,1201,4.6,34.19925,-118.53276,01:00,2019-01-01,09:00,2019-06-24
2,Los Angeles,1201,2.8,34.19925,-118.53276,02:00,2019-01-01,10:00,2019-06-24
3,Los Angeles,1201,4.8,34.19925,-118.53276,03:00,2019-01-01,11:00,2019-06-24
4,Los Angeles,1201,3.0,34.19925,-118.53276,04:00,2019-01-01,12:00,2019-06-24
5,Los Angeles,1201,-1.5,34.19925,-118.53276,05:00,2019-01-01,13:00,2019-06-24
6,Los Angeles,1201,-0.3,34.19925,-118.53276,06:00,2019-01-01,14:00,2019-06-24
7,Los Angeles,1201,4.0,34.19925,-118.53276,07:00,2019-01-01,15:00,2019-06-24
8,Los Angeles,1201,2.5,34.19925,-118.53276,08:00,2019-01-01,16:00,2019-06-24
9,Los Angeles,1201,2.3,34.19925,-118.53276,09:00,2019-01-01,17:00,2019-06-24


In [6]:
california_filtered_df=california_filtered_df.dropna()

In [7]:
california_filtered_df = california_filtered_df[(california_filtered_df[['sample_measurement']] > 0).all(axis=1)]
california_filtered_df.head()

,county,site_number,sample_measurement,latitude,longitude,time_local,date_gmt,time_gmt,date_of_last_change
0,Los Angeles,1201,5.8,34.19925,-118.53276,00:00,2019-01-01,08:00,2019-06-24
1,Los Angeles,1201,4.6,34.19925,-118.53276,01:00,2019-01-01,09:00,2019-06-24
2,Los Angeles,1201,2.8,34.19925,-118.53276,02:00,2019-01-01,10:00,2019-06-24
3,Los Angeles,1201,4.8,34.19925,-118.53276,03:00,2019-01-01,11:00,2019-06-24
4,Los Angeles,1201,3.0,34.19925,-118.53276,04:00,2019-01-01,12:00,2019-06-24


In [8]:
california_filtered_df.describe()

,site_number,sample_measurement,latitude,longitude
count,38401.000000,38401.000000,38401.000000,38401.000000
mean,2391.170647,11.212643,34.111205,-118.257149
std,2175.244013,9.246106,0.192668,0.254512
min,16.000000,0.100000,33.792360,-118.884819
25%,1103.000000,6.100000,34.066590,-118.528400
50%,1201.000000,10.000000,34.144350,-118.226880
75%,4004.000000,14.700000,34.199250,-118.175330
max,9034.000000,995.600000,34.813034,-117.850360


In [9]:
# Set Index
california_filtered_df=california_filtered_df.set_index('date_of_last_change')

# Sort Values
ca_dates = california_filtered_df.sort_values(by='date_of_last_change', axis=0).index.unique()

,county,site_number,sample_measurement,latitude,longitude,time_local,date_gmt,time_gmt
date_of_last_change,,,,,,,,
2019-06-24,Los Angeles,1201,5.8,34.19925,-118.53276,00:00,2019-01-01,08:00
2019-06-24,Los Angeles,1201,4.6,34.19925,-118.53276,01:00,2019-01-01,09:00
2019-06-24,Los Angeles,1201,2.8,34.19925,-118.53276,02:00,2019-01-01,10:00
2019-06-24,Los Angeles,1201,4.8,34.19925,-118.53276,03:00,2019-01-01,11:00
2019-06-24,Los Angeles,1201,3.0,34.19925,-118.53276,04:00,2019-01-01,12:00
...,...,...,...,...,...,...,...,...
2020-03-31,Los Angeles,1103,10.0,34.06659,-118.22688,19:00,2019-12-02,03:00
2020-03-31,Los Angeles,1103,11.0,34.06659,-118.22688,20:00,2019-12-02,04:00
2020-03-31,Los Angeles,1103,9.0,34.06659,-118.22688,21:00,2019-12-02,05:00


## Begin Heat Map. 

In [12]:
# Single out Lat,Lon values to combine into list
lat_cords = [i for i in california_filtered_df['latitude']]
lon_cords = [i for i in california_filtered_df['longitude']]
# Bring out humit values. Used for weights in gmaps
humid_weights = california_filtered_df['sample_measurement'].tolist()

In [13]:
# Go through two lists to create [(lat,lng)] for coordinate values. 
# Used in gmaps
coordinates = []
for i,j in zip(lat_cords, lon_cords):
    coords = (i,j)
    coordinates.append(coords)

In [14]:
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(coordinates, weights=humid_weights, 
                                 dissipating=False, max_intensity=.4,
                                 point_radius=.1)

#markers = gmaps.marker_layer(coordinates)
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

Edited to emphasize weights

In [15]:
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(coordinates, weights=humid_weights, 
                                 dissipating=False, max_intensity=.99,
                                 point_radius=.2)

#markers = gmaps.marker_layer(coordinates)
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…